In [25]:
import pandas as pd
import numpy as np

### Загрузка данных

In [30]:
raw_data = pd.read_csv("data/raw/NASDAQ_FULL_2021_Close.csv")
raw_data.head()

,Date,AAPL,ABBV,ABT,ACN,ADBE,ADP,AMD,AMGN,AMT,...,HLAHU,HLAHW,HLBZW,HLGN-WT,HLLY-WT,HMA-UN,HMA-WT,HMACR,HMACU,HMACW
0,2021-01-04,125.974464,86.739708,99.687309,238.710358,485.339996,153.288055,92.300003,193.642502,191.466125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-05,127.532021,87.636642,100.920723,240.069351,485.690002,152.861603,92.769997,194.582230,192.433868,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-06,123.239037,86.879608,100.710602,242.694153,466.309998,152.326324,90.330002,199.272491,187.359482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-07,127.444374,87.809464,101.688164,244.983932,477.739990,153.188248,95.160004,199.930344,189.635117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-08,128.544388,88.270248,101.971436,245.877502,485.100006,155.184143,94.580002,203.749222,193.733017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
raw_data.drop(columns=["Date"], inplace=True)
raw_data.head()

,AAPL,ABBV,ABT,ACN,ADBE,ADP,AMD,AMGN,AMT,ASML,...,HLAHU,HLAHW,HLBZW,HLGN-WT,HLLY-WT,HMA-UN,HMA-WT,HMACR,HMACU,HMACW
0,125.974464,86.739708,99.687309,238.710358,485.339996,153.288055,92.300003,193.642502,191.466125,477.949158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,127.532021,87.636642,100.920723,240.069351,485.690002,152.861603,92.769997,194.582230,192.433868,483.072815,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,123.239037,86.879608,100.710602,242.694153,466.309998,152.326324,90.330002,199.272491,187.359482,468.600525,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,127.444374,87.809464,101.688164,244.983932,477.739990,153.188248,95.160004,199.930344,189.635117,479.316132,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,128.544388,88.270248,101.971436,245.877502,485.100006,155.184143,94.580002,203.749222,193.733017,486.303741,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Фильтрация активов

In [32]:
raw_data.isna().sum(axis=0).value_counts().sort_index()

0      6324
1        13
2         3
4        72
5         4
       ... 
246       1
247       1
248       2
250       3
252    4727
Name: count, Length: 220, dtype: int64

Удалим те, что имеют более 50 пропусков (n/a). Это те активы, по которым либо не нашлось данных (ещё не обращались на бирже), либо листинг которых произошёл в 2021 году (например, в его середине), и полных данных по ним нет.

In [33]:
MAX_NA_THRESHOLD = 50

In [34]:
na_counts = raw_data.isna().sum(axis=0)
valid_tickers = na_counts[na_counts <= MAX_NA_THRESHOLD].index

data = raw_data[valid_tickers]

In [35]:
print(f"Tickers was: {data.shape[1]}")
print(f"Deleted tickers (NA > {MAX_NA_THRESHOLD}): {raw_data.shape[1] - data.shape[1]}")
print(f"Tickers left for analysis: {data.shape[1]}")

Tickers was: 6561
Deleted tickers (NA > 50): 5289
Tickers left for analysis: 6561


### Заполнение пропусков

In [36]:
data.ffill(inplace=True)
data.bfill(inplace=True)

C:\Users\Yuriy\AppData\Local\Temp\ipykernel_51188\2526927714.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.ffill(inplace=True)
C:\Users\Yuriy\AppData\Local\Temp\ipykernel_51188\2526927714.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.bfill(inplace=True)


Проверим, что мы точно почистили все n/a:

In [37]:
data.isna().sum(axis=0).value_counts()

0    6561
Name: count, dtype: int64

Проверим, нет ли отрицательных цен в данных

In [ ]:
# Удаление активов с аномальными ценами (Отрицательные или Нулевые)
# Цены акций не могут быть отрицательными. Если после Imputation появились отрицательные 
# или нулевые значения, это серьезная ошибка данных. Удаляем такие тикеры.

has_non_positive_price = (data <= 0).any(axis=0)
non_positive_tickers = has_non_positive_price[has_non_positive_price].index.tolist()

print(f"Total tickers with non-positive price found: {len(non_positive_tickers)}")
print(f"Examples of non-positive tickers: {non_positive_tickers[:10]} ...")

initial_columns = data.shape[1]
# Drop columns (tickers) that have at least one non-positive price
data = data.drop(columns=non_positive_tickers, errors='ignore')
deleted_count = initial_columns - data.shape[1]

print(f"Removed {deleted_count} assets with non-positive prices.")
print(f"Tickers left after price cleaning: {data.shape[1]}")

# Финальная проверка чистоты данных
if (data <= 0).any().any():
    print("Error: Non-positive prices remain in data!")
elif data.isna().sum().sum() > 0:
    print("Error: N/A values remain in data!")
else:
    print("Data is clean and ready for return calculation.")


Total tickers with non-positive price found: 21
Examples of non-positive tickers: ['SVA', 'CBIO', 'PLTYF', 'MSVB', 'MSTO', 'AMCCF', 'ASEKF', 'DISPF', 'DQJCF', 'PBNNF'] ...
Removed 21 assets with non-positive prices.
Tickers left after price cleaning: 6540
Data is clean and ready for return calculation.


Экспортируем очищенные данные:

In [39]:
data.to_csv("data/processed/NASDAQ_FULL_2021_Cleaned_Imputed.csv", index=False)